In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def fake_data(n=1000, n_cont=5, n_cat=2, task='class'):
    rng = np.random.default_rng(8)
    cont = rng.standard_normal(size=(n, n_cont))
    cat = rng.integers(3, size=(n, n_cat))
    cont_names = [f'cont_{i}' for i in range(n_cont)]
    cat_names = [f'cat_{i}' for i in range(n_cat)]
    left = pd.DataFrame(cont, columns=cont_names, dtype=np.float32)
    right = pd.DataFrame(cat, columns=cat_names, dtype=np.int)
    df = left.join(right)
    df['targ'] = cont.sum(axis=1) + cat.sum(axis=1) - 1
    if task=='class':
        df['targ'] = (df['targ']>0).astype(np.float32)
    return df, cont_names, cat_names

In [ ]:
df, cont_names, cat_names = fake_data()

In [ ]:
df.head()

,cont_0,cont_1,cont_2,cont_3,cont_4,cat_0,cat_1,targ
0,-1.738266,-1.336643,-1.361107,-0.351617,-2.312582,2,1,0.0
1,-0.188897,-0.957229,0.893600,0.956847,1.392258,0,0,1.0
2,0.767470,-0.053030,0.859794,1.505481,-0.653595,0,1,1.0
3,0.610351,-0.042674,1.440017,-0.836895,-0.301547,2,1,1.0
4,0.362339,0.258110,-1.639448,0.360155,-0.118498,2,1,1.0


In [ ]:
from nn4tab.data import cont_cat_split, TabularDataset, get_dsets

In [ ]:
dep_var = 'targ'

In [ ]:
cont, cat = cont_cat_split(df, dep_var)

In [ ]:
assert cont==cont_names

In [ ]:
assert cat==cat_names

In [ ]:
train_ds, valid_ds = get_dsets(df, cat, cont, [dep_var])

In [ ]:
train_ds[0]

(array([2, 2]),
 array([-0.46469232,  0.6805593 ,  0.9084532 ,  0.9904088 , -0.2751088 ],
       dtype=float32),
 array([1.], dtype=float32))

In [ ]:
train_ds.data[dep_var].mean(), valid_ds.data['targ'].mean()

(0.66625, 0.665)